In [1]:
%load_ext autoreload
%autoreload 2

In [112]:
from abc import ABC, abstractmethod
from collections import ChainMap
from functools import wraps, update_wrapper
import inspect
from inspect import Parameter, getsource, getsourcefile, getfile, ismodule, \
    ismethod, isfunction

from htools import *

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
@contextmanager
def temporary_global_scope(kwargs):
    # Make kwargs temporarily available as global vars.
    print('in ctx', kwargs)
    old_globals = globals().copy()
    globals().update(kwargs)
    try:
        yield
    finally:
        for k in kwargs:
            if k in old_globals:
                globals()[k] = old_globals[k]
            else:
                del globals()[k]

In [101]:
def add_kwargs(*fns, positional=False):
    """When multiple functions contain a parameter with the same name, 
    priority is determined by the order of `fns`. Wrapped function must accept
    **kwargs.
    """
    param_types = [Parameter.POSITIONAL_ONLY,
                   Parameter.POSITIONAL_OR_KEYWORD, 
                   Parameter.KEYWORD_ONLY]
    if not positional: param_types = param_types[1:]
    def _args(fn): 
        res = {}
        for k, v in params(fn).items():
            # If positional=False, allow positional_or_keyword args with
            # defaults but not those without. 
            if v.kind in param_types and (positional 
                                          or v.default != inspect._empty):
                res[k] = v
        return res
    
    # ChainMap operates in reverse order so functions that appear earlier in
    # `fns` take priority.
    extras_ = dict(ChainMap(*map(_args, fns)))

    def decorator(func):
        """First get params present in func's original signature, then get
        params from additional functions which are NOT present in original 
        signature. Combine and sort param lists so positional args come first
        etc. Finally replace func's signature with our newly constructed one.
        """
        sig = inspect.signature(func)
        extras = [v for v in 
                  select(extras_, drop=sig.parameters.keys()).values()]
        parameters = sorted(
            list(sig.parameters.values()) + extras, 
            key=lambda x: (x.kind, x.default != inspect._empty)
        )
        func.__signature__ = sig.replace(parameters=parameters)
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            """Execute wrapped function in a context where kwargs are 
            temporarily available as globals. Globals will be restored to 
            its prior state once execution completes.
            """
            # Order matters here: defaults must come first so user-passed 
            # args/kwargs will override them.
            kwargs = {**{p.name: p.default for p in extras},  
                      **func.__signature__.bind(*args, **kwargs).arguments}
            with temporary_global_scope(kwargs):
                return func(**kwargs)
        return wrapper
    return decorator

In [131]:
class CLI(ABC):
    
    def __init__(self, *funcs):
        self.funcs = list(funcs)
        
        @add_kwargs(*self.funcs, positional=True)
        def _call_wrapper(self, **kwargs):
            print(_call_wrapper.__signature__)
            print(Args(a=a, b=b, c=c, d=d))
            return self.call(**kwargs)
        CLI.__call__ = _call_wrapper
        
    @abstractmethod
    def call(self):
        raise NotImplementedError()

In [132]:
class MyCLI(CLI):
    
    # TODO: mostly works but I don't think we support args/kwargs at the 
    # moment. Maybe add them to signature with func name, i.e. bar_kwargs?
    def call(self, **kwargs):
        print('in __call__', kwargs)
        f_res = foo(a=a, b=b)
        print(f_res)
        b_res = baz(c=c, d=d)
        print(b_res)

In [136]:
def foo(a, b=3):
    return 'foo', {'a': a, 'b': b}

def baz(c, d=6):
    return 'baz', {'c': c, 'd': d}

In [137]:
cli = MyCLI(foo, baz)

In [138]:
cli('c0', 'a1', 'd2')

in ctx {'c': 'c0', 'd': 'd2', 'a': 'a1', 'b': 3, 'self': <__main__.MyCLI object at 0x1a17a29e10>}
(self, c, a, d=6, b=3, **kwargs)
Args(a='a1', b=3, c='c0', d='d2')
in __call__ {'c': 'c0', 'd': 'd2', 'a': 'a1', 'b': 3}
('foo', {'a': 'a1', 'b': 3})
('baz', {'c': 'c0', 'd': 'd2'})


In [110]:
cli = CLI(foo, baz)

In [111]:
cli.bar('x1', 9, d=11, b=44)

in ctx {'c': 'x1', 'd': 11, 'a': 9, 'b': 44, 'self': <__main__.CLI object at 0x1a16171c50>}
(self, c, a, d=6, b=3, **kwargs)
Args(a=9, b=44, c='x1', d=11)
in __call__ {'c': 'x1', 'd': 11, 'a': 9, 'b': 44}
396
x1x1x1x1x1x1x1x1x1x1x1


### Alternate strategies

- Update signature automatically with deco as above, but then explicitly update globals with kwargs. Is this any safer? Maybe not. I guess it removes 1 layer of hackery.
- 